In [ ]:
%%bash
# Install Rust
curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
source "$HOME/.cargo/env"

# Install Python dependencies
python -m pip install --upgrade pip
python -m pip install jupyter numpy matplotlib

# Install AICI
git clone https://github.com/microsoft/aici.git
cd aici
cargo build --release
python -m pip install -e .

# Microsoft AICI - Prompts as (Wasm) Programs

AICI (Artificial Intelligence Controller Interface) is a framework that allows building controllers to constrain and direct output of large language models (LLMs). Key features:

- Flexible - Controllers can be written in languages that compile to WebAssembly (Rust, C, C++, etc) or interpreted inside Wasm (Python, JavaScript, etc)
- Secure - Controllers run sandboxed without access to filesystem, network, etc
- Fast - Wasm modules compile to native code and run in parallel with LLM inference
- Integrates with LLMs like llama.cpp, Hugging Face Transformers, rLLM

AICI aims to simplify development of controllers, allow writing fast controllers, and enable compatibility across LLM engines. It supports local and cloud execution.

## Architecture

AICI consists of:
- AICI Spec - Defines the interface between AICI runtime and AI Controllers 
- AICI Runtime - Rust implementation of the AICI Spec
- AI Controllers - Wasm modules that implement controller logic
  - Can be written in Rust, C/C++, or other languages that compile to Wasm
  - Or use PyCtrl/JsCtrl to write controllers in Python/JavaScript interpreted inside Wasm
- LLM Backends - Interface with various LLMs
  - Currently supports llama.cpp, Hugging Face Transformers, rLLM
  - Planned support for vLLM

## Getting Started

1. Install dependencies (see first code cell)

2. Run the examples
  - Rust examples in `controllers/rust`
  - Python examples using PyCtrl in `controllers/pyctrl`
  - JavaScript examples using JsCtrl in `controllers/jsctrl`

## Writing a Custom Controller

Controllers can implement custom logic to control the LLM generation process:
- Constrained decoding 
- Dynamic editing of prompts and generated text
- Coordinating parallel generations
- Maintaining state during generation

Some example use cases:
- Ensure generated JSON matches a schema
- Implement custom sampling methods 
- Enable multi-agent conversations
- Control information flow and prompts based on previous interactions

In [ ]:
# Example of using PyCtrl to generate a list

import aici

async def main():
    prompt = "What are the most popular types of vehicles?\n"
    await aici.FixedTokens(prompt)
    
    marker = aici.Label()
    for i in range(1,6):
        await aici.FixedTokens(f"{i}.")
        await aici.gen_text(stop_at="\n")
        await aici.FixedTokens("\n")
        
    aici.set_var("result", marker.text_since())

aici.start(main())